# Notes/Ideas



In [6]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
from wordcloud import WordCloud
import pandas as pd
import numpy as np

import json

import requests

from bs4 import BeautifulSoup
import re

import time

import nltk

plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('fivethirtyeight')
plt.style.use('ggplot')

In [7]:
import warnings
warnings.filterwarnings('ignore')

# Faculty Profiles data

In [8]:
df_faculty = pd.read_csv('../scraped_data/faculty_profiles.csv', sep='~')

In [9]:
df_faculty.head()

,faculty_name,faculty_profile_url,l_expertise,department,title_name,faculty_site_url,lab_url,faculty_email,description,description_links,...,link_to_news_3,description_teaser_3,article_date_4,title_of_news_4,link_to_news_4,description_teaser_4,article_date_5,title_of_news_5,link_to_news_5,description_teaser_5
0,David A. Aaker,/faculty/david-aaker,"business,marketing,branding",Haas School of Business,Professor of Marketing and Public Policy,http://www.haas.berkeley.edu/faculty/aaker.html,http://groups.haas.berkeley.edu/marketing/,aaker@haas.berkeley.edu,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pieter Abbeel,/faculty/pieter-abbeel,"robotics,machine learning",Division of Computer Science/EECS,Professor,http://www.cs.berkeley.edu/~pabbeel,NaN,pabbeel@cs.berkeley.edu,Robotics and Machine Learning.,NaN,...,/news/new-deep-learning-technique-enables-robo...,UC Berkeley researchers have developed algor...,"December 17, 2012",Big NSF grant funds research into training ro...,/news/big-nsf-grant-funds-research-training-ro...,"What if robots and humans, working together,...","August 23, 2011",UC Berkeley robotics expert named among world...,/news/uc-berkeley-robotics-expert-named-among-...,"Pieter Abbeel, a UC Berkeley, professor know..."
2,Elizabeth Abel,/faculty/elizabeth-abel,"feminist theory,psychoanalysis,Virginia Woolf,...",Department of English,Professor of English,http://english.berkeley.edu/profiles/5,NaN,eabel@uclink.berkeley.edu,Elizabeth Abel's general research interest is...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dor Abrahamson,/faculty/dor-abrahamson,"mathematical cognition,design-based research,m...",Graduate School of Education,Associate Professor of Cognition and Development,http://gse.berkeley.edu/people/dor-abrahamson,http://edrl.berkeley.edu/,dor@berkeley.edu,Dor Abrahamson studies the process of mathema...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Norman Abrahamson,/faculty/norman-abrahamson,"civil and environmental engineering,earthquake...",Department of Civil and Environmental Engineering,Adjunct Professor of Civil and Environmental E...,http://www.ce.berkeley.edu/faculty/faculty.php...,NaN,naa3@earthlink.net,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_faculty.columns.values

array(['faculty_name', 'faculty_profile_url', 'l_expertise', 'department',
       'title_name', 'faculty_site_url', 'lab_url', 'faculty_email',
       'description', 'description_links', 'article_date_1',
       'title_of_news_1', 'link_to_news_1', 'description_teaser_1',
       'article_date_2', 'title_of_news_2', 'link_to_news_2',
       'description_teaser_2', 'article_date_3', 'title_of_news_3',
       'link_to_news_3', 'description_teaser_3', 'article_date_4',
       'title_of_news_4', 'link_to_news_4', 'description_teaser_4',
       'article_date_5', 'title_of_news_5', 'link_to_news_5',
       'description_teaser_5'], dtype=object)

In [11]:
def split_first_last_name(s):
    """
    Find highest two 'words', those are the first and last name by order
    """
    l = s.lower().split(' ')
    lengths = [len(w) for w in l]
    max_index = max(range(len(l)), key=lambda i:lengths[i])
    max_word = l[max_index]
    lengths[max_index] = -1
    
    max_index2 = max(range(len(l)), key=lambda i:lengths[i])
    max_word2 = l[max_index2]
    
    if max_index < max_index2:
        return max_word, max_word2
    else:
        return max_word2, max_word
    
    

first_last_names = df_faculty['faculty_name'].apply(split_first_last_name)
first_names = [t[0] for t in first_last_names]
last_names = [t[1] for t in first_last_names]
df_faculty['first_name'] = first_names
df_faculty['last_name'] = last_names

In [12]:
df_faculty['full_name'] = df_faculty['first_name'] + ' ' + df_faculty['last_name']

In [13]:
bool_filter = ~df_faculty['l_expertise'].isnull()
df_faculty = df_faculty[bool_filter]
df_faculty['l_expertise'] = df_faculty['l_expertise'].apply(lambda s:s.lower())

In [14]:
len(set(df_faculty['department']))

72

In [15]:
set(df_faculty['department'])

{'Boalt Hall School of Law',
 'College of Engineering',
 'College of Letters and Science',
 'College of Natural Resources',
 'Department of African American Studies',
 'Department of Agricultural & Resource Economics',
 'Department of American Studies',
 'Department of Anthropology',
 'Department of Architecture',
 'Department of Art Practice',
 'Department of Astronomy',
 'Department of Bioengineering',
 'Department of Chemical Engineering',
 'Department of Chemistry',
 'Department of City & Regional Planning',
 'Department of Civil and Environmental Engineering',
 'Department of Classics',
 'Department of Comparative Literature',
 'Department of Demography',
 'Department of Earth and Planetary Science',
 'Department of East Asian Languages and Cultures',
 'Department of Economics',
 'Department of English',
 'Department of Environmental Science, Policy & Management',
 'Department of Ethnic Studies',
 'Department of Film and Media',
 'Department of French',
 "Department of Gender and 

In [141]:
finished_departments = set([
 'College of Engineering',
 'College of Letters and Science',
 'College of Natural Resources',
 'Department of Agricultural & Resource Economics',
 'Department of Astronomy',
 'Department of Bioengineering',
 'Department of Chemical Engineering',
 'Department of Chemistry',
 'Department of City & Regional Planning',
 'Department of Civil and Environmental Engineering',
 'Department of Economics',
 'Department of Environmental Science, Policy & Management',
 'Department of History',
 'Department of Industrial Engineering & Operations Research',
 'Department of Integrative Biology',
 'Department of Materials Science and Engineering',
 'Department of Mathematics',
 'Department of Molecular & Cell Biology',
 'Department of Nuclear Engineering',
 'Department of Nutritional Sciences & Toxicology',
 'Department of Physics',
 'Department of Plant and Microbial Biology',
 'Department of Statistics',
 'Division of Computer Science/EECS',
 'Division of Electribcal Engineering/EECS',
 'Environmental Science, Policy & Management',
 'Department of Earth and Planetary Science',
 'Division of Electrical Engineering/EECS',
 'Haas School of Business',
 'Helen Wills Neuroscience Institute',
 'Energy & Resources Group',
 'Department of Psychology',
 'Department of Political Science',
 'Department of Sociology',
 'Department of Anthropology',
 'Goldman School of Public Policy',
 'Department of Linguistics',
 'School of Public Health',
 'School of Social Welfare',
 'Landscape Architecture & Environmental Planning',
 'School of Information',
])

### Todo list

In [142]:
set(df_faculty['department']) - finished_departments

{'Boalt Hall School of Law',
 'Department of African American Studies',
 'Department of American Studies',
 'Department of Architecture',
 'Department of Art Practice',
 'Department of Classics',
 'Department of Comparative Literature',
 'Department of Demography',
 'Department of East Asian Languages and Cultures',
 'Department of English',
 'Department of Ethnic Studies',
 'Department of Film and Media',
 'Department of French',
 "Department of Gender and Women's Studies",
 'Department of Geography',
 'Department of German',
 'Department of History of Art',
 'Department of Italian Studies',
 'Department of Mechanical Engineering',
 'Department of Music',
 'Department of Near Eastern Studies',
 'Department of Philosophy',
 'Department of Rhetoric',
 'Department of Scandinavian',
 'Department of Slavic Languages and Literatures',
 'Department of South and Southeast Asian Studies',
 'Department of Spanish and Portuguese',
 'Graduate School of Education',
 'Interdisciplinary Studies (ISF

### Departments/Colleges/Institutes that are covered by smaller departments/mix of other departments

'Helen Wills Neuroscience Institute' is made up of mcb and other related departments

'Energy & Resources Group' espm, mcb, etc

### A lot of the faculty profiles don't work or don't point to the right source

In [18]:
df_faculty['faculty_site_url'].values[:30]

array(['http://www.haas.berkeley.edu/faculty/aaker.html',
       'http://www.cs.berkeley.edu/~pabbeel',
       'http://english.berkeley.edu/profiles/5',
       'http://gse.berkeley.edu/people/dor-abrahamson',
       'http://www.ce.berkeley.edu/faculty/faculty.php?id=780',
       'http://history.berkeley.edu/people/richard-m-abrams',
       'http://sph.berkeley.edu/barbara-abrams-drph-rd',
       'http://www.law.berkeley.edu/faculty/profiles/facultyProfile.php?facID=1105',
       'http://ced.berkeley.edu/ced/faculty-staff/charisma-acey', nan,
       'https://gspp.berkeley.edu/directories/faculty/dan-acland',
       'http://cci.lbl.gov/paul',
       'http://optometry.berkeley.edu/faculty/anthony-j-adams',
       'http://math.berkeley.edu/people/faculty/john-w-addison',
       'http://www.ieor.berkeley.edu/People/Faculty/adler.htm',
       'http://physics.berkeley.edu/people/faculty/mina-aganagic',
       'http://anthropology.berkeley.edu/users/sabrina-c-agarwal',
       'http://polisci.b

In [61]:
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}

def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    elif element == '\n':
        return False
    return True

def scrape_urls(url, fn, base_url=''):
    if type(url) == type([]):
        url_l = url
        total_urls = {}
        for url in url_l:
            soup = BeautifulSoup(requests.get(url, headers=headers).content)
            urls = fn(soup)
            for faculty in urls:
                total_urls[faculty] = base_url + urls[faculty]
        return total_urls

    soup = BeautifulSoup(requests.get(url, headers=headers).content)
    urls = fn(soup)
    for faculty in urls:
        urls[faculty] = base_url + urls[faculty]
    return urls

def feed_me_soup(soup, use_div, class_name=None, id_name=None):
    selector = 'div' if use_div else 'section'
    try:
        if id_name is not None:
            dump = list(filter(visible, soup.find_all(selector, {'id':id_name})[0].find_all(text=True)))
            return ' '.join([t for t in [t.replace(u'\xa0', u'').strip() for t in dump] if t != ''])
        dump = list(filter(visible, soup.find_all(selector, {'class':class_name})[0].find_all(text=True)))
        return ' '.join([t for t in [t.replace(u'\xa0', u'').strip() for t in dump] if t != ''])
    except:
        dump = list(filter(visible, soup.find_all(text=True)))
        return ' '.join([t for t in [t.replace(u'\xa0', u'').strip() for t in dump] if t != ''])

def scrape_faculty_page(url, department_name):
    soup = BeautifulSoup(requests.get(url, headers=headers).content)
    use_div = True
    if 'section' in departments_d[department_name]:
        use_div = False
        
    if 'use_content_id' in departments_d[department_name]:
        dump = feed_me_soup(soup, use_div, id_name=departments_d[department_name]['content_fn'])
    else:
        dump = feed_me_soup(soup, use_div, class_name=departments_d[department_name]['content_fn'])
    return dump

In [140]:
def get_school_of_info_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_social_welfare_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'block-system-main'})[0].find_all('div', {'class':'view-content'})[0]\
        .find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_public_health_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'content'})[0].find_all('div', {'class':'view-content'})[0]\
        .find_all('div', recursive=False)
    for div in divs:
        block = div.find_all('div', recursive=False)
        name = block[1].find_all('h3')[0].text
        name = name.replace(',', '').replace('MD', '').replace('DrPH', '')\
                .replace('PhD', '').replace('MA', '')\
                .replace('MS', '').replace('MPH', '').replace('MRP', '').replace('RD', '').strip()
        url = block[3].find_all('a')[0]['href']
        urls[name] = url
    return urls

def get_linguistics_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'main-inner'})[0].find_all('div', {'class':'panel-panel-inner'})[0]\
        .find_all('div', recursive=False)
    for div in divs:
        try:
            a = div.find_all('h2')[0].find_all('a')[0]
            url = a['href']
            if url.startswith('/'):
                url = 'http://lx.berkeley.edu{}'.format(url)
            urls[a.text] = url
        except:
            pass
    return urls

def get_anthro_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'class':'view-content'})[1].find_all('table')[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[0].find_all('div')[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_sociology_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'class':'view-content'})[0].find_all('table')[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[0].find_all('a')[0]
        urls[a.text] = a['href']

    return urls

def get_poli_sci_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'class':'view-content'})[0].find_all('table')[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[0].find_all('a')[0]
        urls[a.text] = a['href']

    return urls

def get_psych_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
    for div in divs:
        block = div.find_all('div', {'class':'panel-panel panel-col'})[0]
        a = block.find_all('a')[0]
        spans = block.find_all('span')
        urls['{} {}'.format(spans[0].text.strip(), spans[1].text.strip())] = a['href']
    return urls

def get_gssp_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'content'})[0].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('h3')[0].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_haas_urls(soup):
    urls = {}
    blocks = soup.find_all('section', {'id':'center-col'})[0].find_all('div', {'class':'faculty-block'})
    for div in blocks:
        divs = div.find_all('div', recursive=False)
        for div in divs:
            a = div.find_all('p')[0].find_all('a')[0]
            last, first = split_first_last_name(a.text.replace('\n', '').replace('\t', '').replace(',', '').strip())
            urls['{} {}'.format(first, last)] = a['href']
    urls['john phillips'] = urls['(jack) phillips']
    del urls['(jack) phillips']
    return urls

def get_nuc_eng_urls(soup):
    urls = {}
    for a in soup.find_all('div', {'class':'view-content'})[0].find_all('a'):
        urls[a.text] = a['href']
    del urls['']
    return urls

def get_mse_urls(soup):
    urls = {}
    tables = soup.find_all('table', {'class':'facultytable'})
    for table in tables:
        tds = table.find_all('td')
        for td in tds:
            try:
                aas = td.find_all('a')
                if len(aas) > 1:
                    a = aas[1]
                else:
                    a = aas[0]
                urls[a.text.strip().replace('\xa0', ' ')] = a['href'].strip()
            except Exception as e:
                pass
    return urls

def get_eps_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        ps = div.find_all('p')
        for p in ps:
            a = p.find_all('a')[0]
            urls[a.text] = a['href'][5:]
    return urls

def get_city_regional_planning_urls(soup):
    urls = {}
    lis = soup.find_all('div', {'id':'faculty'})[0].find_all('li')
    for li in lis:
        urls[li.find_all('h2')[0].text.replace('  ', ' ')] = li.find_all('a')[0]['href']
    return urls

def get_astro_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'row faculty-list researcher-list clearfix'})[0].find_all('div')
    for div in divs:
        a = div.find_all('a')[1]
        urls[a.text] = a['href']
    return urls

def get_plant_micro_bio_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_nutri_sci_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

def get_physics_urls(soup):
    urls = {}
    uls = soup.find_all('div', {'class':'view-content'})[1].find_all('ul')
    for ul in uls:
        lis = ul.find_all('li')
        for li in lis:
            a = li.find_all('a')[1]
            urls[a.text] = a['href']
    return urls

def get_ib_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('a')[0]
        urls[a.text.replace('  ', ' ')] = a['href']
    return urls

def get_env_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text.replace('  ', ' ')] = a['href']
    return urls

def get_are_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'content'})[0].find_all('div', recursive=False)[1:]
    for div in divs:
        subdivs = div.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
        for subdiv in subdivs:
            a = subdiv.find_all('div', recursive=False)[1].find_all('a')[0]
            urls[a.text] = a['href']
    return urls

def get_econ_urls(soup):
    urls = {}
    trs = soup.find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[1].find_all('a')[0]
        last, first = a.text.strip().split(', ')
        urls[first + ' ' + last] = a['href']
    return urls

def get_chem_urls(soup):
    ahrefs = soup.find_all('div', {'class':'content'})[0]\
            .find_all('div', recursive='False')[0].find_all('div', recursive='False')[0]\
                .find_all('div', recursive='False')[0].find_all('a')
    urls = {}
    for a in ahrefs:
        try:
            if a['href'].startswith('http://'):
                last, first = a.text.split(', ')
                urls[first + ' ' + last] = a['href']
        except:
            pass
    return urls

def get_ieor_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'faculty-group clearfix'})
    for div in divs:
        lis = div.find_all('ol')[0].find_all('li', recursive=False)
        for li in lis:
            a = li.find_all('div')[2].find_all('a')[0]
            urls[a.text] = a['href']
    urls['J George Shanthikumar'] = urls[' \tJ George Shanthikumar ']
    del urls[' \tJ George Shanthikumar ']
    return urls

def get_civil_env_e_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div')
    for div in divs:
        try:
            a = div.find_all('div')[1].find_all('a')[0]
            urls[a.text.replace('  ', ' ')] = a['href']
        except:
            pass
    return urls

def get_bioe_urls(soup):
    urls = {}
    ahrefs = soup.find_all('div', {'class':'entry-content'})[0].find_all('a')
    for a in ahrefs:
        urls[a.find_all('strong')[0].text] = a['href']
    return urls

def get_mcb_urls(soup):
    urls = {}
    ahrefs = soup.find_all('div', {'id':'block-system-main'})[0].find_all('a')
    for a in ahrefs:
        try:
            url = a['href']
            if url.startswith('/'):
                urls[a.text] = url
        except:
            pass
    del urls['Faculty by Division']
    del urls['More Information >']
    return urls

def get_statistics_urls(soup):
    urls = {}
    tds = soup.find_all('tbody')[0].find_all('td')
    for td in tds:
        try:
            a = td.find_all('div')[2].find_all('a')
            urls[a[0].text] = a[0]['href']
        except:
            pass
    return urls

def get_math_urls(soup):
    urls = {}
    tds = soup.find_all('tbody')[0].find_all('td')
    for td in tds:
        try:
            a = td.find_all('div')[1].find_all('a')
            urls[a[0].text] = a[0]['href']
        except:
            pass
    return urls

def get_eecs_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'media'})
    for div in divs:
        a = div.find_all('a')
        urls[a[1].text] = a[0]['href']
    return urls

def get_history_urls(soup):
    urls = {}
    trs = soup.find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('a')[0]
        urls[a.text] = a['href']
    return urls

In [135]:
departments_d = {
    'history': {
        'url':'http://history.berkeley.edu/faculty',
        'base':'http://history.berkeley.edu',
        'url_fn':get_history_urls,
        'content_fn':'region region-content'
    },
    'eecs': {
        'url':'https://www2.eecs.berkeley.edu/Faculty/Lists/list.html?_ga=1.58368611.858746805.1441508156',
        'base':'https://www2.eecs.berkeley.edu',
        'url_fn':get_eecs_urls,
        'content_fn':'content'
    },
    'math': {
        'url':'https://math.berkeley.edu/people/faculty',
        'base':'https://math.berkeley.edu',
        'url_fn':get_math_urls,
        'content_fn':'content'
    },
    'stats': {
        'url':'http://statistics.berkeley.edu/people/faculty',
        'base':'http://statistics.berkeley.edu',
        'url_fn':get_statistics_urls,
        'content_fn':'content-wrapper',
        'use_content_id':True
    },
    'mcb': {
        'url':'https://mcb.berkeley.edu/faculty/all',
        'base':'https://mcb.berkeley.edu',
        'url_fn':get_mcb_urls,
        'content_fn':'content',
        'use_content_id':True
    },
    'bioe': {
        'url':'http://bioeng.berkeley.edu/people',
        'url_fn':get_bioe_urls,
        'content_fn':'entry-content'
    },
    'civil_env_e': {
        'url':'http://www.ce.berkeley.edu/people/faculty',
        'base':'http://www.ce.berkeley.edu',
        'url_fn':get_civil_env_e_urls,
        'content_fn':'view-content'
    },
    'ieor': {
        'url':'http://ieor.berkeley.edu/people/faculty',
        'base':'http://ieor.berkeley.edu',
        'url_fn':get_ieor_urls,
        'content_fn':'content',
        'use_content_id':True
    },
    'chem': {
        'url':'http://chemistry.berkeley.edu/faculty/chem',
        'url_fn':get_chem_urls,
        'content_fn':'content'
    },
    'cheme': {
        'url':'http://chemistry.berkeley.edu/faculty/cbe',
        'url_fn':get_chem_urls,
        'content_fn':'content'
    },
    'econ': {
        'url': 'https://www.econ.berkeley.edu/faculty',
        'url_fn':get_econ_urls,
        'content_fn':'view-content'
    },
    'are': {
        'url':'https://are.berkeley.edu/people/faculty',
        'base':'https://are.berkeley.edu',
        'url_fn':get_are_urls,
        'content_fn':'view-content'
    },
    'env': {
        'url':'https://ourenvironment.berkeley.edu/people/faculty-specialists',
        'base':'https://ourenvironment.berkeley.edu',
        'url_fn':get_env_urls,
        'content_fn':'view-content'
    },
    'ib': {
        'url':'http://ib.berkeley.edu/research/interests',
        'base':'https://ib.berkeley.edu',
        'url_fn':get_ib_urls,
        'content_fn':'node-content'
    },
    'astro_physics': {
        'url':'http://physics.berkeley.edu/research/astrophysics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'molecular_physics': {
        'url':'http://physics.berkeley.edu/research/atomic-molecular-and-optical-physics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'biophysics': {
        'url':'http://physics.berkeley.edu/research/biophysics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'condensed_matter_physics': {
        'url':'http://physics.berkeley.edu/research/condensed-matter-physics-and-materials-science',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'nuclear_physics': {
        'url':'http://physics.berkeley.edu/research-faculty/nuclear-physics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'particle_physics': {
        'url':'http://physics.berkeley.edu/research/particle-physics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'plasma_physics': {
        'url':'http://physics.berkeley.edu/research/plasma-and-nonlinear-dynamics',
        'base':'http://physics.berkeley.edu',
        'url_fn':get_physics_urls,
        'content_fn':'bio-content'
    },
    'nutri_sci': {
        'url':'http://nst.berkeley.edu/people/faculty',
        'base':'http://nst.berkeley.edu',
        'url_fn':get_nutri_sci_urls,
        'content_fn':'view-content'
    },
    'plant_micro_bio': {
        'url':'http://plantandmicrobiology.berkeley.edu/faculty/directory?_ga=1.25912820.858746805.1441508156',
        'base':'http://plantandmicrobiology.berkeley.edu',
        'url_fn':get_plant_micro_bio_urls,
        'content_fn':'entity entity-profile2 profile2-main clearfix'
    },
    'astro': {
        'url':'http://astro.berkeley.edu/people/researchers',
        'base':'http://astro.berkeley.edu',
        'url_fn':get_astro_urls,
        'content_fn':'detail-body'
    },
#     'city_regional_planning': {
#         'url':'https://ced.berkeley.edu/academics/city-regional-planning/people/faculty/',
#         'url_fn':get_city_regional_planning_urls,
#         'content_fn':'profile-content'
#     },
    'eps': {
        'url':'http://eps.berkeley.edu/research',
        'base':'http://eps.berkeley.edu/people',
        'url_fn':get_eps_urls,
        'content_fn':'ds-2col-stacked user-profile view-mode-full clearfix'
    },
    'mse': {
        'url':'https://www.mse.berkeley.edu/people/faculty',
        'url_fn':get_mse_urls,
        'content_fn':'entry-content'
    },
    'nuc_eng': {
        'url':'http://www.nuc.berkeley.edu/people/faculty',
        'base':'http://www.nuc.berkeley.edu',
        'url_fn':get_nuc_eng_urls,
        'content_fn':'content'
    },
    'haas': {
        'url':'http://facultybio.haas.berkeley.edu/faculty-photo/',
        'url_fn':get_haas_urls,
        'content_fn':'center-col',
        'use_content_id':True,
        'section':True
    },
    'gssp': {
        'url':'https://gspp.berkeley.edu/directories/faculty',
        'url_fn':get_gssp_urls,
        'content_fn':'content',
        'use_content_id':True,
    },
    'psych': {
        'url':'http://psychology.berkeley.edu/people/faculty',
        'base':'http://psychology.berkeley.edu/',
        'url_fn':get_psych_urls,
        'content_fn':'card-content',
    },
    'poli_sci': {
        'url':['http://polisci.berkeley.edu/people/faculty', 'http://polisci.berkeley.edu/people/faculty?page=1'],
        'base':'http://polisci.berkeley.edu',
        'url_fn':get_poli_sci_urls,
        'content_fn':'profile',
    },
    'sociology': {
        'url':'http://sociology.berkeley.edu/regular-faculty',
        'base':'http://sociology.berkeley.edu',
        'url_fn':get_sociology_urls,
        'content_fn':'profile',
    },
    'anthro': {
        'url':'http://anthropology.berkeley.edu/faculty',
        'base':'http://anthropology.berkeley.edu',
        'url_fn':get_anthro_urls,
        'content_fn':'user-profile-view',
        'use_content_id':True
    },
    'linguistics': {
        'url':'http://lx.berkeley.edu/people/faculty',
        'url_fn':get_linguistics_urls,
        'content_fn':'main-inner',
        'use_content_id':True
    },
    'public_health': {
        'url':['http://sph.berkeley.edu/faculty-profiles']+\
            ['http://sph.berkeley.edu/faculty-profiles?title=&page={}'.format(i) for i in range(1, 6)],
        'base':'http://sph.berkeley.edu',
        'url_fn':get_public_health_urls,
        'content_fn':'content',
        'use_content_id':True
    },
    'social_welfare': {
        'url':['http://socialwelfare.berkeley.edu/faculty-profiles',
            'http://socialwelfare.berkeley.edu/academic-researchers'],
        'base':'http://socialwelfare.berkeley.edu',
        'url_fn':get_social_welfare_urls,
        'content_fn':'block-system-main',
        'use_content_id':True
    },
    'ced': {
        'url':'https://ced.berkeley.edu/ced/faculty-staff/',
        'url_fn':get_city_regional_planning_urls,
        'content_fn':'profile-content'
    },
    'school_of_info': {
        'url':['https://www.ischool.berkeley.edu/people?name=&role=122'] + \
        ['https://www.ischool.berkeley.edu/people?name=&role=122&degr=All&year[value]&spec=All&emp=&page={}'.format(i)
            for i in range(1, 4)],
        'base':'https://www.ischool.berkeley.edu',
        'url_fn':get_school_of_info_urls,
        'content_fn':'main-content',
        'use_content_id':True
    },
}

In [60]:
for department in departments_d:
    d = departments_d[department]
    if 'base' in d:
        urls = scrape_urls(d['url'], d['url_fn'], d['base'])
    else:
        urls = scrape_urls(d['url'], d['url_fn'])
    d['urls'] = urls

In [33]:
def get_urls(department):
    return departments_d[department]['urls']

In [163]:
text_d = {}
for department_name in departments_d:
    print('Starting: {}'.format(department_name))
    urls_d = get_urls(department_name)
    for faculty in urls_d:
        url = urls_d[faculty]
        try:
            text_d[faculty] = scrape_faculty_page(url, department_name)
        except Exception as e:
            print(department_name)
            print(e)

Starting: nuclear_physics
Starting: cheme
Starting: condensed_matter_physics
Starting: plant_micro_bio
Starting: history
Starting: env
Starting: ib
Starting: nuc_eng
Starting: psych
Starting: stats
Starting: city_regional_planning
Starting: nutri_sci
Starting: math
Starting: biophysics
Starting: astro
Starting: eps
eps
list index out of range
eps
list index out of range
eps
list index out of range
eps
list index out of range
eps
list index out of range
eps
list index out of range
eps
list index out of range
eps
list index out of range
Starting: poli_sci
Starting: econ
Starting: ieor
Starting: sociology
Starting: gssp
Starting: eecs
Starting: are
Starting: civil_env_e
Starting: astro_physics
astro_physics
Exceeded 30 redirects.
Starting: bioe
Starting: particle_physics
Starting: mcb
Starting: mse
mse
list index out of range
Starting: chem
Starting: haas
Starting: plasma_physics
Starting: molecular_physics


In [164]:
rows = []
for faculty in text_d:
    rows.append([faculty, text_d[faculty]])
profile_df = pd.DataFrame(rows, columns=['faculty', 'text'])
profile_df.to_csv('../scraped_data/faculty_website_profiles.csv', sep='~', index=False)

In [165]:
len(profile_df)

1697

In [166]:
profile_df = pd.read_csv('../scraped_data/faculty_website_profiles.csv', sep='~')

In [35]:
urls = get_urls('plant_micro_bio')
urls.keys()

dict_keys(['Russell Jones', 'Devin Coleman-Derr', 'Patricia C. Zambryski', 'Shauna Somerville', 'Jennifer Fletcher', 'Tom Bruns', 'Bob B. Buchanan', 'Peggy G. Lemaux', 'Sheila McCormick', 'Matt F. Traxler', 'Dominique  Loqué', 'Igor Grigoriev', 'Barbara Baker', 'Z. Renee Sung', 'Kathleen Ryan', 'Mary C. Wildermuth', 'Arash Komeili', 'Lewis J. Feldman', 'Michael R. Freeling', 'John W.  Taylor', 'Frank G. Harmon', 'Sarah C. Hake', 'Steven E. Lindow', 'Kimberley Seed ', 'Jennifer D. Lewis', 'Brian Staskawicz', 'Chelsea D. Specht', 'Peter Quail', 'Chris R. Somerville', 'John  Vogel', 'Norman Terry', 'Daniel Zilberman', 'Steven E. Brenner', 'Henrik Vibe Scheller', 'Rachel Brem', 'Kimmen Sjölander', 'Markus Pauly', 'Michi Taga', 'Sheng Luan', 'Britt A. Glaunsinger', 'Robert Fischer', 'N. Louise Glass', 'Benjamin Blackman', 'Cheryl A. Kerfeld', 'Anastasios Melis', 'John D. Coates', 'Krishna K. Niyogi', 'Andrew O. Jackson'])

In [37]:
urls['Barbara Baker']

'http://plantandmicrobiology.berkeley.edu/profile/bbaker'

In [41]:
profile_df[profile_df['faculty'] == 'Barbara Baker']['text'].values

array([ "Barbara Baker Plant Innate Immunity Adjunct Professor 800 Buchanan Street Albany, CA  94710 Phone 510.559.5912 Lab Phone 510.559.5931 Fax 510.642.4995 bbaker@berkeley.edu http://bakerlab.berkeley.edu Research Recent Publications Recent Teaching Ph.D.Microbiology and ImmunologyUCSF, 1981 B.S.BiologyUCSD, 1974 Research The Baker Lab seeks understanding of the molecular, genetic and biochemical bases of host-microbe interactions, and investigates mechanisms of pathogen-induced host disease and disease resistance. Our goal is to understand the biochemical and molecular bases of plant resistance to pathogen diseases. We especially are interested in understanding the mechanism of plant-pathogen recognition and signal transduction leading to the induction of disease resistance responses. Our experimental system consists of the tobacco mosaic virus (TMV) resistance gene, N, encoding a putative cytoplasmic receptor, disease-resistant plant hosts harboring N, and the classical plant pat

## Ignore: for adding new departments/testing

In [233]:
set(df_faculty['department'])

{'Boalt Hall School of Law',
 'College of Engineering',
 'College of Letters and Science',
 'College of Natural Resources',
 'Department of African American Studies',
 'Department of Agricultural & Resource Economics',
 'Department of American Studies',
 'Department of Anthropology',
 'Department of Architecture',
 'Department of Art Practice',
 'Department of Astronomy',
 'Department of Bioengineering',
 'Department of Chemical Engineering',
 'Department of Chemistry',
 'Department of City & Regional Planning',
 'Department of Civil and Environmental Engineering',
 'Department of Classics',
 'Department of Comparative Literature',
 'Department of Demography',
 'Department of Earth and Planetary Science',
 'Department of East Asian Languages and Cultures',
 'Department of Economics',
 'Department of English',
 'Department of Environmental Science, Policy & Management',
 'Department of Ethnic Studies',
 'Department of Film and Media',
 'Department of French',
 "Department of Gender and 

In [158]:
gspp_url = 'https://gspp.berkeley.edu/directories/faculty'
psych_url = 'http://psychology.berkeley.edu/people/faculty'
base_url = 'http://psychology.berkeley.edu/'
poli_sci_url = 'http://polisci.berkeley.edu/people/faculty'
base = 'http://polisci.berkeley.edu'

sociology_url = 'http://sociology.berkeley.edu/regular-faculty'

anthro_url = 'http://anthropology.berkeley.edu/faculty'
linguistics_urls = 'http://lx.berkeley.edu/people/faculty'

public_health_urls = 'http://sph.berkeley.edu/faculty-profiles'

social_welfare_urls = ['http://socialwelfare.berkeley.edu/faculty-profiles',
            'http://socialwelfare.berkeley.edu/academic-researchers']

school_of_info_urls = ['https://www.ischool.berkeley.edu/people?name=&role=122'] + \
        ['https://www.ischool.berkeley.edu/people?name=&role=122&degr=All&year[value]&spec=All&emp=&page={}'.format(i)
            for i in range(1, 4)]
    
school_of_optometry_urls = 'https://optometry.berkeley.edu/people/'

headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
r = requests.get(school_of_optometry_urls, headers=headers)
soup = BeautifulSoup(r.content)

In [162]:
def get_sociology_urls(soup):
    urls = {}
    trs = soup.find_all('div', {'class':'view-content'})[0].find_all('table')[0].find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[0].find_all('a')[0]
        urls[a.text] = a['href']

    return urls

def get_social_welfare_urls(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'block-system-main'})[0].find_all('div', {'class':'view-content'})[0]\
        .find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

#school of optometry hard codes data in javascript or either renders the data into javascript instead
# of directly rendering into html
# to make data miners lives more difficult?
#soup.find_all('article', {'class':'grid-people row'})[0].find_all('section')
#soup

In [49]:
urls

{'aaron bodoh-creed': 'http://facultybio.haas.berkeley.edu/faculty-list/bodohcreed-aaron/',
 'aaron mcdaniel': 'http://facultybio.haas.berkeley.edu/faculty-list/mcdaniel-aaron/',
 'abby scott': 'http://facultybio.haas.berkeley.edu/faculty-list/scott-abby/',
 'abhishek nagaraj': 'http://facultybio.haas.berkeley.edu/faculty-list/nagaraj-abhishek/',
 'adair morse': 'http://facultybio.haas.berkeley.edu/faculty-list/morse-adair/',
 'adam berman': 'http://facultybio.haas.berkeley.edu/faculty-list/berman-adam/',
 'adam leipzig': 'http://facultybio.haas.berkeley.edu/faculty-list/leipzig-adam/',
 'ajay bam': 'http://facultybio.haas.berkeley.edu/faculty-list/bam-ajay/',
 'alan cerf': 'http://facultybio.haas.berkeley.edu/faculty-list/cerf-alan/',
 'alan ross': 'http://facultybio.haas.berkeley.edu/faculty-list/ross-alan/',
 'alastair lawrence': 'http://facultybio.haas.berkeley.edu/faculty-list/lawrence-alastair/',
 'alexander nezlobin': 'http://facultybio.haas.berkeley.edu/faculty-list/nezlobin-al

In [60]:
faculty_url = 'http://facultybio.haas.berkeley.edu/faculty-list/berman-adam/'
r = requests.get(faculty_url, headers=headers)
soup = BeautifulSoup(r.content)

In [137]:
department = department_name = 'school_of_info'

d = departments_d[department]
if 'base' in d:
    urls = scrape_urls(d['url'], d['url_fn'], d['base'])
else:
    urls = scrape_urls(d['url'], d['url_fn'])
d['urls'] = urls

text_d = {}
urls_d = get_urls(department_name)
for faculty in urls_d:
    url = urls_d[faculty]
    try:
        text_d[faculty] = scrape_faculty_page(url, department_name)
    except Exception as e:
        print(department_name)
        print(e)

In [138]:
text_d.keys()

dict_keys(['Mark Mims', 'Paul Duguid', 'David Reiley', 'Geoffrey Nunberg', 'Annette Greiner', 'Jenna Burrell', 'Gunnar Kleemann', 'Deirdre Mulligan', 'D. Alex Hughes', 'Pamela Samuelson', 'Zachary Alexander', 'Daniel Gillick', 'Steve Fadden', 'Edward Fine', 'Kevin Crook', 'Chris Jay Hoofnagle', 'Mary Kay Duggan', 'Doug Tygar', 'Divya Ramachandran', 'Lisa Prescott', 'Joyce Shen', 'Christopher Llop', 'James Shanahan', 'Todd Holloway', 'Michael Cooper', 'Brad DesAulniers', 'Zachary Pardos', 'Nancy Van House', 'Xiao Qiang', 'David Bamman', 'Kurt Beyer', 'Justin Erlich', 'Arash Nourian', 'Brooks Ambrose', 'Jari Koister', 'Coco Krumme', 'Anna Lauren Hoffmann', 'Nathaniel Good', 'Robert Youmans', 'Alberto Todeschini', 'Alex Marrs', 'James Kunz', 'John Alexis Guerra Gómez', 'Esteban Arias Navarro', 'Andreas Weigend', 'James Pierce', 'Jez Humble', 'Michael Buckland', 'Amit Bhattacharyya', 'David Steier', 'Ryan Kappedal', 'Marti Hearst', 'Ray Larson', 'Paul Laskowski', 'Robert Glushko', 'Yoonjun

In [149]:
text_d[list(text_d.keys())[1]]

'Paul Duguid Adjunct Professor Focus The concept of information; the history of information; the history of trademarks and their informational role; critical information studies. Research areas Social & Cultural Studies Biography Current Research Information and authenticity. From a historical perspective: how we come to trust information we encounter and how we led credibility to information we share and what role brands and other forms of certification play. Other Research Visiting Fellow, Lancaster University Management School Education BA, English & Philosophy, Bristol University, UK, 1976 MA, English Literature, Washington University, St. Louis, 1980 What is the best thing about working at the I School? Multidisciplinary faculty and students. What Information issues interests you most? Questions of authenticity of information. <a class="addthis_button_facebook" title="Facebook"></a>\n  <a class="addthis_button_twitter" title="Twitter"></a>\n  <a class="addthis_button_google_pluson

### categories of text

- keywords/research fields
- description/bio
- selected publications

In [194]:
departments_d['bioe']['content_fn'] = 'entry-content'

In [142]:
list(departments_d.keys())

['cheme',
 'bioe',
 'are',
 'biophysics',
 'stats',
 'env',
 'econ',
 'ib',
 'nutri_sci',
 'history',
 'ieor',
 'math',
 'civil_env_e',
 'condensed_matter_physics',
 'plant_micro_bio',
 'eecs',
 'astro_physics',
 'nuclear_physics',
 'mcb',
 'chem',
 'molecular_physics',
 'particle_physics',
 'plasma_physics']

In [36]:
department_name = 'eecs'

In [37]:
urls = get_urls(department_name)
urls

{'Abhay Parekh': 'https://www2.eecs.berkeley.edu/Faculty/Homepages/parekh.html',
 'Adam Wolisz': 'https://www2.eecs.berkeley.edu/Faculty/Homepages/wolisz.html',
 'Alberto L. Sangiovanni-Vincentelli': 'https://www2.eecs.berkeley.edu/Faculty/Homepages/sangiovanni-vicentelli.html',
 'Alessandro Chiesa': 'https://www2.eecs.berkeley.edu/Faculty/Homepages/alexch.html',
 'Alexandra von Meier': 'https://www2.eecs.berkeley.edu/Faculty/Homepages/vonmeier.html',
 'Alexandre Bayen': 'https://www2.eecs.berkeley.edu/Faculty/Homepages/bayen.html',
 'Alexei (Alyosha) Efros': 'https://www2.eecs.berkeley.edu/Faculty/Homepages/efros.html',
 'Ali Ghodsi': 'https://www2.eecs.berkeley.edu/Faculty/Homepages/aligh.html',
 'Ali Javey': 'https://www2.eecs.berkeley.edu/Faculty/Homepages/javey.html',
 'Ali Niknejad': 'https://www2.eecs.berkeley.edu/Faculty/Homepages/niknejad.html',
 'Alistair Sinclair': 'https://www2.eecs.berkeley.edu/Faculty/Homepages/sinclair.html',
 'Ana Claudia Arias': 'https://www2.eecs.berk

In [39]:
i = 0
print(urls[list(urls.keys())[i]])
scrape_faculty_page(urls[list(urls.keys())[i]], department_name)

https://www2.eecs.berkeley.edu/Faculty/Homepages/ramchandran.html


'Kannan Ramchandran Professor Info Links Research Areas Communications & Networking (COMNET) Signal Processing (SP) Control, Intelligent Systems, and Robotics (CIR) Research Centers Berkeley Wireless Research Center (BWRC) Berkeley Laboratory for Information and System Sciences (BLISS) Simons Institute for the Theory of Computing (SITC) Berkeley Audio-visual Signal processing and Communication Systems (BASiCS) Teaching Schedule Spring 2017 EE 126. Probability and Random Processes, TuTh 9:30AM - 10:59AM, Cory 277 Biography He received his B.E. from the City College of New York in\n1982, and his M.S. and Ph.D. from Columbia University in 1984 and 1993, respectively. After completing his Ph.D., Professor Ramchandran became an Assistant Professor at the University of Illinois, Urbana-Champaign and Associate Director of the Beckman Institute Multimedia Laboratory.  In the Beckman Institute, he was a member of the Image Formation and Processing Group.  In 1999, he came to UC Berkeley\'s EECS

In [217]:
dump = scrape_faculty_page(urls[list(urls.keys())[i]], department_name)

In [226]:
text = nltk.Text(nltk.tokenize.word_tokenize(dump))

In [227]:
text.collocations()

lattice potentials; technological importance; Bose-Einstein
condensate; Physical Review; Review Letters; spinor Bose; atomic
physics; quantum physics


In [228]:
text.vocab()

FreqDist({"''": 2,
          '(': 15,
          ')': 15,
          ',': 62,
          '.': 36,
          '0.02': 1,
          '045305': 1,
          '108': 1,
          '11': 1,
          '143201': 1,
          '1486': 1,
          '1936': 1,
          '1999': 1,
          '2000': 2,
          '2001': 2,
          '2002': 2,
          '2003': 1,
          '2005': 1,
          '2006': 1,
          '2007': 2,
          '2012': 2,
          '2014': 1,
          '2015': 1,
          '312': 1,
          '344': 1,
          '433': 1,
          '720': 1,
          '95': 1,
          ':': 2,
          'A': 1,
          'APS': 1,
          'Alfred': 1,
          'America': 1,
          'American': 1,
          'And': 1,
          'Andrew': 1,
          'Another': 1,
          'Area': 1,
          'Ashvin': 1,
          'Atomic': 2,
          'Atoms': 1,
          'Award': 1,
          'Barrios': 1,
          'Berkeley': 1,
          'Biography': 1,
          'Bose': 2,
          'Bose-Einstein'

In [218]:
nltk.tokenize.word_tokenize(dump)

['Research',
 'Area',
 '(',
 's',
 ')',
 ':',
 'Atomic',
 ',',
 'Molecular',
 'and',
 'Optical',
 'Physics',
 'Condensed',
 'Matter',
 'Physics',
 'And',
 'Materials',
 'Science',
 'Biography',
 'Dan',
 'M.',
 'Stamper-Kurn',
 'came',
 'to',
 'Berkeley',
 'following',
 'his',
 'studies',
 'at',
 'the',
 'Massachusetts',
 'Institute',
 'of',
 'Technology',
 '(',
 'Ph.D.',
 '2000',
 ')',
 'and',
 'postdoctoral',
 'work',
 'at',
 'the',
 'California',
 'Institute',
 'of',
 'Technology',
 '(',
 '1999',
 '–',
 '2001',
 ')',
 '.',
 'He',
 'is',
 'the',
 'recipient',
 'of',
 'the',
 '2000',
 'APS',
 'Division',
 'of',
 'Atomic',
 ',',
 'Optical',
 'and',
 'Molecular',
 'Physics',
 'Outstanding',
 'Thesis',
 'award',
 ',',
 'the',
 'Alfred',
 'P.',
 'Sloan',
 'Fellowship',
 '(',
 '2001',
 '–',
 '2003',
 ')',
 ',',
 'the',
 'David',
 'and',
 'Lucile',
 'Packard',
 'Fellowship',
 'in',
 'Science',
 'and',
 'Engineering',
 '(',
 '2002',
 '–',
 '2007',
 ')',
 ',',
 'and',
 'the',
 'Presidential',


In [98]:
' '.join([t for t in [t.replace(u'\xa0', u'').replace("\'", "'").strip() for t in dump] if t != ''])

'Real name: Maureen C. Miller Professor Office Hours: Tuesday, 9-10; Thursday, 3-4 2219 Dwinelle (510) 642-5678 mcmiller@berkeley.edu Education: Ph.D., Harvard University, 1989 M.A. Catholic University of America, 1983 B.A. The American University, 1981 Curriculum Vitae: Miller_CV.pdf Research Interests: My research and teaching explores the extraordinary capacity of individuals and societies for change, using the rapid transformation of Europe over the eleventh and twelfth centuries as a lense to interogate contemporary understandings of life choices and the social, economic, political, and cultural forces conditioning them. Medieval religious beliefs and practices figure centrally in my work: I am profoundly interested in the radical claims made for human efficacy on the basis of otherworldly premises and the paradoxical results of attempts to realize religious ideals in this world. I have a particular interest in movements for reform in the eleventh and twelfth centuries. What reall

In [38]:
results = []
l_links_to_follow = []
count = 1
for url in urls:
    if count % 10 == 0:
        print(count)
    count+=1
    
    try:
        faculty_soup = BeautifulSoup(requests.get(base_url+url).content)
        #ahrefs = faculty_soup.find_all('a')
        #l = [ahref['href'] for ahref in ahrefs]
        #links_to_follow.append(l) #go one deep
        result = feed_me_soup(faculty_soup)
        results.append(result)
    except:
        results.append([])
        
    time.sleep(0.1)

10
20
30
40
50
60
70
80
90
100
110
120
130


In [39]:
results[1]

['Maneesh Agrawala',
 'Adjunct Professor',
 'Info Links',
 'Research Areas',
 'Human-Computer Interaction (HCI)',
 'Research Centers',
 'Berkeley Center for New Media (BCNM)',
 'Berkeley Vision and Learning Center (BVLC)',
 'Biography',
 "\n                        Maneesh Agrawala is an Adjunct Professor in Electrical Engineering and Computer Science at the University of California, Berkeley.  He is currently a Professor of Computer Science and Director of the Brown Institute for Media Innovation at Stanford University.  He works on visualization, computer graphics and human computer interaction. His focus is on investigating how cognitive design principles can be used to improve the effectiveness of visual displays. The goals of this work are to discover the design principles and then instantiate them in both interactive and automated design tools. He received an Okawa Foundation Research Grant in 2006, an Alfred P. Sloan Foundation Fellowship and an NSF CAREER Award in 2007, a SIGGRA

In [12]:
# r = requests.get('https://www2.eecs.berkeley.edu/Faculty/Lists/list.html?_ga=1.58368611.858746805.1441508156')
# soup = BeautifulSoup(r.content)
# base_url = 'https://www2.eecs.berkeley.edu'
# urls = []
# divs = soup.find_all('div', {'class':'media'})
# names = []
# for div in divs:
#     a = div.find_all('a')
#     urls.append(a[0]['href'])
#     names.append(a[1].text)
    
urls = df_faculty['faculty_site_url']
    
results = []
l_links_to_follow = []
count = 1
for url in urls:
    if count % 100 == 0:
        print(count)
    count+=1
    
    try:
        faculty_soup = BeautifulSoup(requests.get(url).content)
        ahrefs = faculty_soup.find_all('a')
        l = [ahref['href'] for ahref in ahrefs]
        links_to_follow.append(l) #go one deep
        result = feed_me_soup(faculty_soup)
        results.append(result)
    except:
        results.append([])
        
    time.sleep(0.1)

/home/dust/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/dust/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500


KeyboardInterrupt: 

In [ ]:
for base_url, links_to_follow in zip(url, l_links_to_follow):
    for link in links_to_follow:
        if link.startswith('/'):
            feed_me_soup(BeautifulSoup(request.get(base_url + link).content))
        else:
            feed_me_soup(BeautifulSoup(request.get(link).content)


In [89]:
's'.startswith

<function str.startswith>

In [82]:
list(results[10])

['Skip to main content',
 ' Page Header ',
 'Toggle navigation',
 'EECS at UC Berkeley',
 ' ',
 ' /#name-and-slogan ',
 ' /.navbar-header ',
 'Main menu',
 'About',
 'Toggle submenu for About',
 'About Overview',
 'By the Numbers',
 'Diversity',
 'History',
 'Visiting',
 'Academics',
 'Toggle submenu for Academics',
 'Academics Overview',
 'Undergraduate Admissions & Programs',
 'Graduate Admissions & Programs',
 'Courses',
 'Research',
 'Toggle submenu for Research',
 'Research Overview',
 'Areas',
 'Centers & Labs',
 'Projects',
 'BEARS Symposium',
 'Colloquium',
 'People',
 'Toggle submenu for People',
 'People Overview',
 'Directory',
 'Leadership',
 'Faculty',
 'Students',
 'Staff',
 'Alumni',
 'Connect',
 'Toggle submenu for Connect',
 'Connect Overview',
 'Support EECS',
 'K-12 Outreach',
 'Recruit Students',
 'Contact',
 'Secondary menu',
 'Home',
 'For Students',
 'For Faculty/Staff',
 'Industry',
 'News',
 'Events',
 'Give',
 ' ',
 'Toggle Search',
 'Search form',
 'Search ',

In [ ]:
history_url = 'http://history.berkeley.edu/faculty'
history_base = 'http://history.berkeley.edu'
eecs_url = 'https://www2.eecs.berkeley.edu/Faculty/Lists/list.html?_ga=1.58368611.858746805.1441508156'
eecs_base = 'https://www2.eecs.berkeley.edu'
math_url = 'https://math.berkeley.edu/people/faculty'
math_base = 'https://math.berkeley.edu'
math_content = '#content-area'
stats_url = 'http://statistics.berkeley.edu/people/faculty'
stats_base = 'http://statistics.berkeley.edu'
stats_content = '#content'
mcb_url = 'https://mcb.berkeley.edu/faculty/all'
mcb_base = 'https://mcb.berkeley.edu'
bioe_url = 'http://bioeng.berkeley.edu/people'
bioe_base = ''
civil_env_e_url = 'http://www.ce.berkeley.edu/people/faculty'
civil_env_e_base = 'http://www.ce.berkeley.edu'
ieor_url = 'http://ieor.berkeley.edu/people/faculty'
ieor_base = 'http://ieor.berkeley.edu'
chem_url = 'http://chemistry.berkeley.edu/faculty/chem'
cheme_url = 'http://chemistry.berkeley.edu/faculty/cbe'
econ_url = 'https://www.econ.berkeley.edu/faculty'
are_url = 'https://are.berkeley.edu/people/faculty'
are_base = 'https://are.berkeley.edu'
env_url = 'https://ourenvironment.berkeley.edu/people/faculty-specialists'
env_base = 'https://ourenvironment.berkeley.edu'
ib_url = 'http://ib.berkeley.edu/research/interests'
ib_base = 'https://ib.berkeley.edu'
astro_physics_url = 'http://physics.berkeley.edu/research/astrophysics'
physics_base = 'http://physics.berkeley.edu'
molecular_physics_url = 'http://physics.berkeley.edu/research/atomic-molecular-and-optical-physics'
bio_physics_url = 'http://physics.berkeley.edu/research/biophysics'
condensed_matter_physics_url = 'http://physics.berkeley.edu/research/condensed-matter-physics-and-materials-science'
nuclear_physics_url = 'http://physics.berkeley.edu/research-faculty/nuclear-physics'
particle_physics_url = 'http://physics.berkeley.edu/research/particle-physics'
plasma_physics_url = 'http://physics.berkeley.edu/research/plasma-and-nonlinear-dynamics'
nutri_sci_url = 'http://nst.berkeley.edu/people/faculty'
nutri_sci_base = 'http://nst.berkeley.edu'
plant_micro_bio_url = 'http://plantandmicrobiology.berkeley.edu/faculty/directory?_ga=1.25912820.858746805.1441508156'
plant_micro_bio_base = 'http://plantandmicrobiology.berkeley.edu'

In [15]:
def d_of_plant_micro_bio(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls

plant_micro_bio_urls = get_urls(plant_micro_bio_url, d_of_plant_micro_bio, nutri_sci_base)

In [16]:
def d_of_nutri_sci(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text] = a['href']
    return urls
nutri_sci_urls = get_urls(nutri_sci_url, d_of_nutri_sci, nutri_sci_base)

In [17]:
def d_of_physics(soup):
    urls = {}
    uls = soup.find_all('div', {'class':'view-content'})[1].find_all('ul')
    for ul in uls:
        lis = ul.find_all('li')
        for li in lis:
            a = li.find_all('a')[1]
            urls[a.text] = a['href']
    return urls
astro_phsyics_urls = get_urls(astro_physics_url, d_of_physics, physics_base)

In [18]:
molecular_phsyics_urls = get_urls(molecular_physics_url, d_of_physics, physics_base)

In [19]:
bio_physics_urls = get_urls(bio_physics_url, d_of_physics, physics_base)

In [20]:
condensed_matter_physics_urls = get_urls(condensed_matter_physics_url, d_of_physics, physics_base)

In [21]:
nuclear_physics_urls = get_urls(nuclear_physics_url, d_of_physics, physics_base)

In [22]:
particle_physics_urls = get_urls(particle_physics_url, d_of_physics, physics_base)

In [23]:
plasma_physics_urls = get_urls(plasma_physics_url, d_of_physics, physics_base)

In [24]:
def d_of_ib(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('a')[0]
        urls[a.text.replace('  ', ' ')] = a['href']
    return urls
ib_urls = get_urls(ib_url, d_of_ib, ib_base)

In [25]:
def d_of_env(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div', recursive=False)
    for div in divs:
        a = div.find_all('div', recursive=False)[1].find_all('a')[0]
        urls[a.text.replace('  ', ' ')] = a['href']
    return urls
env_urls = get_urls(env_url, d_of_env, env_base)

In [26]:
def d_of_are(soup):
    urls = {}
    divs = soup.find_all('div', {'id':'content'})[0].find_all('div', recursive=False)[1:]
    for div in divs:
        subdivs = div.find_all('div', {'class':'view-content'})[0].find_all('div', recursive=False)
        for subdiv in subdivs:
            a = subdiv.find_all('div', recursive=False)[1].find_all('a')[0]
            urls[a.text] = a['href']
    return urls
are_urls = get_urls(are_url, d_of_are, are_base)

In [27]:
def d_of_econ(soup):
    urls = {}
    trs = soup.find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('td')[1].find_all('a')[0]
        last, first = a.text.strip().split(', ')
        urls[first + ' ' + last] = a['href']
    return urls
econ_urls = get_urls(econ_url, d_of_econ)

In [29]:
def d_of_chem(soup):
    ahrefs = soup.find_all('div', {'class':'content'})[0]\
        .find_all('div', recursive='False')[0]\
            .find_all('div', recursive='False')[0]\
                .find_all('div', recursive='False')[0]\
                    .find_all('a')
    urls = {}
    for a in ahrefs:
        try:
            if a['href'].startswith('http://'):
                last, first = a.text.split(', ')
                urls[first + ' ' + last] = a['href']
        except:
            pass
    return urls
chem_urls = get_urls(chem_url, d_of_chem)

In [30]:
cheme_urls = get_urls(cheme_url, d_of_chem)

In [32]:
def d_of_ieor(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'faculty-group clearfix'})
    for div in divs:
        lis = div.find_all('ol')[0].find_all('li', recursive=False)
        for li in lis:
            a = li.find_all('div')[2].find_all('a')[0]
            urls[a.text] = a['href']
    return urls

ieor_urls = get_urls(ieor_url, d_of_ieor, ieor_base)
ieor_urls['J George Shanthikumar'] = ieor_urls[' \tJ George Shanthikumar ']
del ieor_urls[' \tJ George Shanthikumar ']

In [33]:
def d_of_civil_env_e(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'view-content'})[1].find_all('div')
    for div in divs:
        try:
            a = div.find_all('div')[1].find_all('a')[0]
            urls[a.text.replace('  ', ' ')] = a['href']
        except:
            pass
    return urls

civil_env_e_urls = get_urls(civil_env_e_url, d_of_civil_env_e, civil_env_e_base)

In [34]:
def d_of_bioe(soup):
    urls = {}
    ahrefs = soup.find_all('div', {'class':'entry-content'})[0].find_all('a')
    for a in ahrefs:
        urls[a.find_all('strong')[0].text] = a['href']
    return urls

bioe_urls = get_urls(bioe_url, d_of_bioe, bioe_base)

In [35]:
def d_of_mcb(soup):
    urls = {}
    ahrefs = soup.find_all('div', {'id':'block-system-main'})[0].find_all('a')
    for a in ahrefs:
        try:
            url = a['href']
            if url.startswith('/'):
                urls[a.text] = url
        except:
            pass
    return urls

mcb_urls = get_urls(mcb_url, d_of_mcb, mcb_base)
del mcb_urls['Faculty by Division']
del mcb_urls['More Information >']

In [36]:
def d_of_statistics(soup):
    urls = {}
    tds = soup.find_all('tbody')[0].find_all('td')
    for td in tds:
        try:
            a = td.find_all('div')[2].find_all('a')
            urls[a[0].text] = a[0]['href']
        except:
            pass
    return urls

stats_urls = get_urls(stats_url, d_of_statistics, stats_base)

In [37]:
def d_of_math(soup):
    urls = {}
    tds = soup.find_all('tbody')[0].find_all('td')
    for td in tds:
        try:
            a = td.find_all('div')[1].find_all('a')
            urls[a[0].text] = a[0]['href']
        except:
            pass
    return urls

math_urls = get_urls(math_url, d_of_math, math_base)

In [38]:
def d_of_eecs(soup):
    urls = {}
    divs = soup.find_all('div', {'class':'media'})
    for div in divs:
        a = div.find_all('a')
        urls[a[1].text] = a[0]['href']
    return urls

eecs_urls = get_urls(eecs_url, d_of_eecs, eecs_base)

In [39]:
def d_of_history(soup):
    urls = {}
    trs = soup.find_all('tbody')[0].find_all('tr')
    for tr in trs:
        a = tr.find_all('a')[0]
        urls[a.text] = a['href']
    return urls

history_urls = get_urls(history_url, d_of_history, history_base)